In [6]:
import pandas as pd
import numpy as np
import pyeeg
from numpy.random import randn
from pandas import ExcelWriter
from openpyxl import Workbook

def rle(inarray):
        """ run length encoding. Partial credit to R rle function. 
            Multi datatype arrays catered for including non Numpy
            returns: tuple (runlengths, startpositions, values) """
        ia = np.array(inarray)                  # force numpy
        n = len(ia)
        if n == 0: 
            return (None, None, None)
        else:
            y = np.array(ia[1:] != ia[:-1])     # pairwise unequal (string safe)
            i = np.append(np.where(y), n - 1)   # must include last element posi
            z = np.diff(np.append(-1, i))       # run lengths
            p = np.cumsum(np.append(0, z))[:-1] # positions
            return(z, p, ia[i])

names1 = [u'Николаева Анна',
u'Мухаметдинов Гаяз',
u'Магнитов Михаил',
u'Пономарев Евгений',
u'Ишханян Аршак']
names2 = [
u'Исрафилов Ильдар',
u'Рулев Георгий',
u'Биктайров Роман',
u'Шилов Иннокентий',
u'Лескин Иван']
#folder = '2016-03-11_Collective_action'
state = 'After_Soc'
label = 2
wb = Workbook()
wb.save(filename = '{}_EEH.xlsx'.format(state))
for line in [1,2]:
    for n in range(5):
        if line ==1 : x = pd.read_csv('{}/{}/000{}_X'.format(state,line,3*n), skiprows=10, sep=')', names = ['time','X'],index_col = False)
        else: x = pd.read_csv('{}/{}/000{}_X'.format(state,line,2*n), skiprows=10, sep=')', names = ['time','X'],index_col = False)

        if line == 1: y = pd.read_csv('{}/{}/000{}_Y'.format(state,line,3*n), skiprows=10, sep=')', names = ['time','Y'],index_col = False)
        else: y = pd.read_csv('{}/{}/000{}_Y'.format(state,line,2*n), skiprows=10, sep=')', names = ['time','Y'],index_col = False)
        if line == 1: z = pd.read_csv('{}/{}/000{}_Z'.format(state,line,3*n+1), skiprows=10, sep=')',names = ['time','Z'],index_col = False)
        if line == 2: z = pd.read_csv('{}/{}/000{}_PZ'.format(state,line,2*n+1), skiprows=10, sep=')',names = ['time','Z'],index_col = False)


        data = x.merge(y).merge(z)
        
        marker = pd.read_excel('{}/1/Markers_1-{}.xls'.format(state,label))
        marker = marker[np.isfinite(marker['sec'])]
        data = data.iloc[:int(marker['tic-tot'].iloc[[-1]])+5000]
        
        df = data[['X','Y','Z']].rolling( window=3, min_periods=1,center=True).mean().diff()
        df = df[np.isfinite(df['X'])]
        df = df**2
        df = np.array(df.sum(axis = 1))
        ndf = []
        prev = 0

        for index in np.array(marker["tic-tot"]):
            if index>df.shape[0]:break
            ndf.append(np.array(df[prev:int(index)]))
            prev = int(index)

        if index<df.shape[0]: ndf.append(np.array(df[int(np.array(marker["tic-tot"])[-1]):]))

        mean = [j.mean() for j in ndf]
        std = [j.std() for j in ndf]
        energy = (np.array(mean)**2+np.array(std))**(1./2)
        hurstl = [pyeeg.hurst(j) for j in ndf]
        
        de = data[['X','Y','Z']].rolling( window=3, min_periods=1,center=True).mean().diff()
        de = de[np.isfinite(de['X'])]
        nde = []
        prev = 0

        for index in np.array(marker["tic-tot"]):
            if index>df.shape[0]:break
            nde.append(np.array(de[prev:int(index)]))
            prev = int(index)

        if index<df.shape[0]: nde.append(np.array(de[int(np.array(marker["tic-tot"])[-1]):]))

        amax = [np.amax(j,axis = 0) for j in nde]
        amin = [np.amin(j,axis = 0) for j in nde]
        delta = (np.array(amax) - np.array(amin))/40
        nde = [(nde[j] - amin[j])//delta[j]+1 for j in range(len(amin))]
        t = [j[:,0]*1000000+j[:,1]*1000+j[:,2] for j in nde]
        entropy = [-sum((rle(np.sort(i))[0]/float(len(i)))*np.log2(rle(np.sort(i))[0]/float(len(i)))) for i in t]
        
        
        
        from openpyxl import load_workbook

        book = load_workbook('{}_EEH.xlsx'.format(state))
        writer = pd.ExcelWriter('{}_EEH.xlsx'.format(state), engine='openpyxl') 
        writer.book = book
        writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

        data10 = pd.DataFrame(list(marker["Unnamed: 1"])).T.to_excel(writer, sheet_name = u'Energy', startrow = 0, startcol = 2,header = False)
        data20 = pd.DataFrame(list(marker["Unnamed: 1"])).T.to_excel(writer, sheet_name = u'Entropy', startrow = 0, startcol = 2,header = False)
        data30 = pd.DataFrame(list(marker["Unnamed: 1"])).T.to_excel(writer, sheet_name = u'Hurst', startrow = 0, startcol = 2,header = False)
        if (line == 1) : 
            data1 = pd.DataFrame(energy, columns=[names1[n]]).T.to_excel(writer, sheet_name = u'Energy', startrow = (line-1)*5+n+1, startcol = 1,header = False)
            data2 = pd.DataFrame(entropy, columns=[names1[n]]).T.to_excel(writer, sheet_name = u'Entropy', startrow = (line-1)*5+n+1, startcol = 1,header = False)
            data3 = pd.DataFrame(hurstl, columns=[names1[n]]).T.to_excel(writer, sheet_name = u'Hurst', startrow = (line-1)*5+n+1, startcol = 1,header = False)
        else:
            data1 = pd.DataFrame(energy, columns=[names2[n]]).T.to_excel(writer, sheet_name = u'Energy', startrow = (line-1)*5+n+1, startcol = 1,header = False)
            data2 = pd.DataFrame(entropy, columns=[names2[n]]).T.to_excel(writer, sheet_name = u'Entropy', startrow = (line-1)*5+n+1, startcol = 1,header = False)
            data3 = pd.DataFrame(hurstl, columns=[names2[n]]).T.to_excel(writer, sheet_name = u'Hurst', startrow = (line-1)*5+n+1, startcol = 1,header = False)
        
        writer.save()

In [ ]:
import pandas as pd
import numpy as np
import pyeeg
from numpy.random import randn
from pandas import ExcelWriter
from openpyxl import Workbook
state = 'After_Soc'
label = 2
for line in [1]:
    for n in range(1):
        if line ==1 : x = pd.read_csv('{}/{}/000{}_X'.format(state,line,3*n), skiprows=10, sep=')', names = ['time','X'],index_col = False)
        else: x = pd.read_csv('{}/{}/000{}_X'.format(state,line,2*n), skiprows=10, sep=')', names = ['time','X'],index_col = False)

        if line == 1: y = pd.read_csv('{}/{}/000{}_Y'.format(state,line,3*n), skiprows=10, sep=')', names = ['time','Y'],index_col = False)
        else: y = pd.read_csv('{}/{}/000{}_Y'.format(state,line,2*n), skiprows=10, sep=')', names = ['time','Y'],index_col = False)
        if line == 1: z = pd.read_csv('{}/{}/000{}_Z'.format(state,line,3*n+1), skiprows=10, sep=')',names = ['time','Z'],index_col = False)
        if line == 2: z = pd.read_csv('{}/{}/000{}_PZ'.format(state,line,2*n+1), skiprows=10, sep=')',names = ['time','Z'],index_col = False)


        data = x.merge(y).merge(z)
        
        marker = pd.read_excel('{}/1/Markers_1-{}.xls'.format(state,label))
        marker = marker[np.isfinite(marker['sec'])]

In [3]:
marker

,[Main],Unnamed: 1,3,tic-tot,sec-tot,tic,min,sec,t,"Δt, sec"
4,Position=1562,ГЗ,[Marker 1],1562.0,31.0,12.0,0.0,31.0,0:31.12,31.0
7,Position=3049,ГЗ_end,[Marker 2],3049.0,60.0,49.0,1.0,0.0,1:0.49,29.0
10,Position=4648,1,[Marker 3],4648.0,92.0,48.0,1.0,32.0,1:32.48,32.0
13,Position=5657,1,[Marker 4],5657.0,113.0,7.0,1.0,53.0,1:53.7,21.0
16,Position=6304,2,[Marker 5],6304.0,126.0,4.0,2.0,6.0,2:6.4,13.0
19,Position=7550,2,[Marker 6],7550.0,151.0,0.0,2.0,31.0,2:31.0,25.0
22,Position=8182,3,[Marker 7],8182.0,163.0,32.0,2.0,43.0,2:43.32,12.0
25,Position=9072,3,[Marker 8],9072.0,181.0,22.0,3.0,1.0,3:1.22,18.0
28,Position=9713,4,[Marker 9],9713.0,194.0,13.0,3.0,14.0,3:14.13,13.0
31,Position=10608,4,[Marker 10],10608.0,212.0,8.0,3.0,32.0,3:32.8,18.0


In [4]:
int(marker['tic-tot'].iloc[[-1]])

42790

In [ ]:
data.iloc[:int(marker['tic-tot'].iloc[[-1]])]